In [1]:
from dataset_io import *
from color_channels import *
import os
import cv2
import numpy as np
import random as ra
from tqdm import tqdm
from profiler import *
import random

## Gradient method

In [2]:
nans=[np.NaN,np.NaN,np.NaN]

def include_border(rect):
    ((x1,y1),(x2,y2))=rect
    border=IMG_SAFETY_BORDER
    x1+=border
    x2+=border
    y1+=border
    y2+=border
    return ((x1,y1),(x2,y2))

def add_border(image):
    border=IMG_SAFETY_BORDER
    return cv2.copyMakeBorder(image, border, border, border, border, cv2.BORDER_REFLECT)

def update_optims(rect):
    ((x1,y1),(x2,y2))=rect
    w,h=(abs(x1-x2), abs(y1-y2))
    global w_max,h_max,w_min,h_min
    w_max=max(w,w_max)
    h_max=max(h,h_max)
    w_min=min(w,w_min)
    h_min=min(h,h_min) 
    
def draw_holes(img):
    h,w,_=img.shape
    hole_1=((x1,y1),(x2,y2))=((int(0.126*w), int(0.3056*h)),(int(0.409*w), int(h)))
    img[y1:y2, x1:x2]=nans
    hole_2=((x1,y1),(x2,y2))=((int(0.5906*w), int(0.3056*h)),(int(0.875*w), int(h)))
    img[y1:y2, x1:x2]=nans

def extract_pallet_rectangles_from_image(rects, imgs, filename, scene_name):
    for j in range(len(rects)):
        rect=rects[j]
        rect=include_border(rect)
        rect=correct_rect_ratio(rect)
        ((x1,y1),(x2,y2))=add_margin(rect)
        cropped_imgs = [img[y1:y2, x1:x2] for img in imgs]
        cropped_imgs = [cv2.resize(img, (WIN_W,WIN_H)) for img in cropped_imgs]
        save_image(cropped_imgs[0], scene_name, filename, "pallet_rectangles_gradient",str(j)+"_ch1",True)
        save_image(cropped_imgs[1], scene_name, filename, "pallet_rectangles_gradient",str(j)+"_ch2",True)
        global pallet_rectangles_gradient
        pallet_rectangles_gradient.append(cropped_imgs)
            
def extract_pallet_color_channels_from_image(rects, imgs, filename, scene_name, pallet_color):
    colors=[pc for pc in pallet_color.split(',')]
    for j in range(len(rects)):
        rect=rects[j]
        (x1,y1),(x2,y2)=rect
        cropped_imgs = [img[y1:y2, x1:x2] for img in imgs]
        [draw_holes(img) for img in cropped_imgs]
        save_image(cropped_imgs[0], scene_name, filename, "pallet_rectangles_color",str(j)+"_ch1",True)
        save_image(cropped_imgs[1], scene_name, filename, "pallet_rectangles_color",str(j)+"_ch2",True)
        global pallets_color
        flat_imgs = [np.reshape(img, (img.shape[0]*img.shape[1],img.shape[2])) for img in cropped_imgs]
        fst_img=flat_imgs[0]
        colorful_part=~(fst_img==0).all(axis=1)
        flat_imgs = [img[colorful_part] for img in flat_imgs]
        
        color=colors[0] if len(colors)==1 else colors[j]
        pallets_color[color].append(flat_imgs)
        
def convert_to_gradient_channels(image):
    img = cv2.UMat(image)
    img = cv2.GaussianBlur(img, (3,3), sigmaX=0, sigmaY=0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
    gradX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1 )
    gradY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1 )
    mag, ori = cv2.cartToPolar(gradX, gradY, angleInDegrees=True)

    ori_ranges = [(0,60), (60,120), (120,180), (180,240), (240,300), (300,360)]
    oris=[cv2.inRange(ori,l,r) for (l,r) in ori_ranges]
    mags= [cv2.copyTo(mag, ori_x) for ori_x in oris]
    channels=[gray,mag]+mags
    channels=[cv2.UMat.get(ch) for ch in channels]
    channels=np.moveaxis(channels, 0, -1)
    return channels

def convert_to_color_channels(image):
    channels=to_color_channels(image, "HSCMYb")
    return channels
    
def save_channels(channels, filename, scene_name, output_dir_name):
    ch1,ch2=np.array_split(channels, 2,2)
    save_image(ch1, scene_name, filename, output_dir_name,"ch1",True)
    save_image(ch2, scene_name, filename, output_dir_name,"ch2",True)

def extract_background_rectangles_from_image(rects, imgs, filename, scene_name):
    if rects:
        sample_pallet_w=0
        y1s=[]
        y2s=[]
        hs=[]
        for j in range(len(rects)):
            rect=rects[j]
            rect=correct_rect_ratio(rect)
            ((x1,y1),(x2,y2))=add_margin(rect)
            w,h=(abs(x1-x2), abs(y1-y2))
            y1s.append(y1)
            y2s.append(y2)
            h=abs(y1-y2)
            hs.append(h)
        h=int(np.mean(hs))
        w=5*h
        y1=min(y1s)
        y2=max(y2s)
        
        img_h,img_w,_=imgs[0].shape
        sampling_x1=0
        sampling_x2=img_w-w
        sampling_upper_y1=0
        sampling_upper_y2=y1-h
        sampling_lower_y1=y2
        sampling_lower_y2=img_h-h
        
        can_sample_upper=sampling_upper_y2>0
        can_sample_lower=sampling_lower_y2>sampling_lower_y1
        
        samples_count=4
        if (can_sample_upper or can_sample_lower) and sampling_x2>0: 
            for i in range(samples_count):
                sampled_x=ra.randint(sampling_x1, sampling_x2)
                sampling_done=False
                if i%2 and can_sample_upper:
                    sampled_y=ra.randint(sampling_upper_y1, sampling_upper_y2)
                    sampling_done=True
                elif can_sample_lower:
                    sampled_y=ra.randint(sampling_lower_y1, sampling_lower_y2)
                    sampling_done=True
                if sampling_done:
                    samples=[img[sampled_y:sampled_y+h, sampled_x:sampled_x+w] for img in imgs]
                    samples=[cv2.resize(sample, (WIN_W,WIN_H)) for sample in samples]
                    save_image(samples[0], scene_name, filename, "backgrounds_gradient",str(i)+"_ch1",True)
                    save_image(samples[1], scene_name, filename, "backgrounds_gradient",str(i)+"_ch2",True)
                    global background_rectangles_gradient
                    background_rectangles_gradient.append(samples)
                   
def natural(num):
    return max(0, num)
                
def extract_background_color_channels_from_image(rects, imgs, filename, scene_name):
    if rects:
        y1s=[]
        y2s=[]
        x1s=[]
        x2s=[]
        for j in range(len(rects)):
            rect=rects[j]
            rect=correct_rect_ratio(rect)
            ((x1,y1),(x2,y2))=add_margin(rect)
            y1s.append(y1)
            y2s.append(y2)
            x1s.append(x1)
            x2s.append(x2)
        y1=natural(min(y1s))
        y2=natural(max(y2s))
        x1=natural(min(x1s))
        x2=natural(max(x2s))
        img_h,img_w,_=imgs[0].shape
        
        
        upper_background=[img[0:y1, x1:x2] for img in imgs]
        lower_background=[img[y2:img_h, x1:x2] for img in imgs]
        backgrounds=[upper_background, lower_background]
        global backgrounds_color
        
        for i in range(len(backgrounds)):
            background=backgrounds[i]
#             save_image(background[0], scene_name, filename, "backgrounds_color",str(i)+"_ch1",True)
#             save_image(background[1], scene_name, filename, "backgrounds_color",str(i)+"_ch2",True)
            background = [np.reshape(img, (img.shape[0]*img.shape[1],img.shape[2])) for img in background]
            indices=range(len(background[0]))
            subsample_indices=random.sample(indices, int(len(indices)/8)) #8 is manually adjusted to my RAM
            background = [img[subsample_indices] for img in background]
            backgrounds_color.append(background)

### Highest level functions

In [3]:
def process_image_optims_extraction(filename, scene_name, label_resolution, rects, pallet_color):
    for rect in rects:
        ((x1,y1),(x2,y2))=rect
        rects=include_border(rect)
        rects=correct_rect_ratio(rects)
        update_optims(rects)

def process_image_gradient_channels_extraction(filename, scene_name, label_resolution, rects, pallet_color):
    image=imread_resized(scene_name, filename, label_resolution)
    channels=convert_to_gradient_channels(image)
    save_channels(channels, filename, scene_name, "channels_gradient")
    
def process_image_color_channels_extraction(filename, scene_name, label_resolution, rects, pallet_color):
    image=imread_resized(scene_name, filename, label_resolution)
    channels=convert_to_color_channels(image)
    save_channels(channels, filename, scene_name, "channels_color")
    
def process_image_gradient_extraction(filename, scene_name, label_resolution, rects, pallet_color):
    channels=read_split_channels(scene_name, filename, "channels_gradient")
    channels_bordered=[add_border(ch) for ch in channels]
    extract_pallet_rectangles_from_image(rects, channels_bordered, filename, scene_name)
    extract_background_rectangles_from_image(rects, channels, filename, scene_name)
    
def process_image_color_extraction(filename, scene_name, label_resolution, rects, pallet_color):
    channels=read_split_channels(scene_name, filename, "channels_color")
#     extract_pallet_color_channels_from_image(rects, channels, filename, scene_name, pallet_color)
    extract_background_color_channels_from_image(rects, channels, filename, scene_name)

def process_data(fun):
    [fun(*row) for row in tqdm(walk_dataset())]    

### Main

In [4]:
IMG_SAFETY_BORDER=130

w_max=h_max=0
w_min=h_min=99999

pallets_color={color: [] for color in COLORS}
backgrounds_color=[]
background_rectangles_gradient=[]
pallet_rectangles_gradient=[]

MODE_GRADIENT_CHANNELS_EXTRACTION=process_image_gradient_channels_extraction
MODE_COLOR_CHANNELS_EXTRACTION=process_image_color_channels_extraction
MODE_GRADIENT_EXTRACTION=process_image_gradient_extraction
MODE_COLOR_EXTRACTION=process_image_color_extraction
MODE_OPTIMS=process_image_optims_extraction
MODE=MODE_GRADIENT_EXTRACTION

In [5]:
profiled('process_data(MODE)', globals(), locals())

100%|██████████| 2092/2092 [01:39<00:00, 20.98it/s]


In [6]:
os.chdir(PROJECT_PATH) 

if MODE==MODE_GRADIENT_EXTRACTION:
    np.save("background_rectangles_gradient",background_rectangles_gradient)
    np.save("pallet_rectangles_gradient",pallet_rectangles_gradient)
    print(np.array(background_rectangles_gradient).shape)
    print(np.array(pallet_rectangles_gradient).shape)
elif MODE==MODE_COLOR_EXTRACTION:
    np.save("backgrounds_color",backgrounds_color)
#     for color in COLORS:
#         np.save("pallets_color_"+color,pallets_color[color])
#         print(np.array(pallets_color[color]).shape)
    print(np.array(backgrounds_color).shape)
elif MODE==MODE_OPTIMS:
    print("w_max: ",w_max,"h_max: ",h_max)
    print("w_min: ",w_min,"h_min: ",h_min)

(7170, 2, 22, 120, 4)
(2591, 2, 22, 120, 4)
